# Tuning BM25 parameters

We tune BM25 parameters on a per-field basis including doc2query expansions and bigrammed text fields. These values are used later when optimizing more complex queries.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import importlib
import os
import sys

from copy import deepcopy
from elasticsearch import Elasticsearch
from skopt.plots import plot_objective

In [3]:
# project library
sys.path.insert(0, os.path.abspath('..'))

import qopt
importlib.reload(qopt)

from qopt.notebooks import evaluate_mrr100_dev_templated, optimize_bm25_mrr100_templated, set_bm25_params
from qopt.optimize import Config, set_bm25_parameters

In [4]:
# use a local Elasticsearch or Cloud instance (https://cloud.elastic.co/)
# es = Elasticsearch('http://localhost:9200')
es = Elasticsearch('http://35.246.228.72:9200')

# set the parallelization parameter `max_concurrent_searches` for the Rank Evaluation API calls
max_concurrent_searches = 30

index = 'msmarco-document.doc2query'
template_id = 'query'

# no query params
query_params = {}

# field names
field_names = [
    'url',
    'title', 'title.bigrams',
    'body', 'body.bigrams',
    'expansions', 'expansions.bigrams'
]
similarity_names = [f"bm25-{x.replace('.', '-')}" for x in field_names]
similarity_name_by_field = { field: similarity for field, similarity in zip(field_names, similarity_names) }

# default Elasticsearch BM25 params
default_bm25_params = {'k1': 1.2, 'b': 0.75}

# base template for tuning
base_templates = [{
    "id": template_id,
    "template": {
        "lang": "mustache",
        "source": { "query": {} }
    }
}]

def reset_all():
    for similarity in similarity_names:
        set_bm25_parameters(es, index, name=similarity, **default_bm25_params)

def for_all_fields(message, fn, existing_results=None):
    _results = {}
    for field, similarity in zip(field_names, similarity_names):
        print(f"{message}: {field}")
        if not existing_results:
            params = default_bm25_params
        else:
            params = existing_results[field][1]
        set_bm25_parameters(es, index, name=similarity, **params)

        _templates = deepcopy(base_templates)
        _templates[0]['template']['source']['query']['match'] = { field: { "query": "{{query_string}}" } }
        _results[field] = fn(_templates, similarity)
    return _results

## Baseline evaluation

In [5]:
%%time

_ = for_all_fields(
    "Dev set evaluation",
    fn=lambda templates, similarity: evaluate_mrr100_dev_templated(es, max_concurrent_searches, index, templates, template_id, query_params),
)

Dev set evaluation: url
Evaluation with: MRR@100
Score: 0.2094
Dev set evaluation: title
Evaluation with: MRR@100
Score: 0.2298
Dev set evaluation: title.bigrams
Evaluation with: MRR@100
Score: 0.1295
Dev set evaluation: body
Evaluation with: MRR@100
Score: 0.2568
Dev set evaluation: body.bigrams
Evaluation with: MRR@100
Score: 0.2015
Dev set evaluation: expansions
Evaluation with: MRR@100
Score: 0.3081
Dev set evaluation: expansions.bigrams
Evaluation with: MRR@100
Score: 0.2837
CPU times: user 15.5 s, sys: 4.3 s, total: 19.8 s
Wall time: 13min 45s


## Optimization

In [6]:
%%time

results = for_all_fields(
    "Optimization",
    fn=lambda templates, similarity: optimize_bm25_mrr100_templated(es, max_concurrent_searches, index, templates, template_id, query_params,
        config_space=Config.parse({
            'method': 'bayesian',
            'num_iterations': 40,
            'num_initial_points': 20,
            'space': {
                'k1': { 'low': 0.0, 'high': 5.0 },
                'b': { 'low': 0.3, 'high': 1.0 },
            }
        }),
        name=similarity),
)

Optimization: url
Optimizing parameters
 - metric: MRR@100
 - queries: data/msmarco-document-sampled-queries.1000.tsv
 - queries: data/msmarco/document/msmarco-doctrain-qrels.tsv
 > iteration 2/40, took 0:00:26 (remains: 0:16:43)
   | 0.2013 (best: 0.2013) - {'k1': 0.9, 'b': 0.4}
 > iteration 3/40, took 0:00:07 (remains: 0:04:37)
   | 0.2014 (best: 0.2014) - {'k1': 0.7028724774489332, 'b': 0.9456519563693777}
 > iteration 4/40, took 0:00:07 (remains: 0:04:28)
   | 0.1717 (best: 0.2014) - {'k1': 3.380479439302227, 'b': 0.7408976496526426}
 > iteration 5/40, took 0:00:07 (remains: 0:04:18)
   | 0.1991 (best: 0.2014) - {'k1': 0.8582755849004388, 'b': 0.9913057071684106}
 > iteration 6/40, took 0:00:08 (remains: 0:04:54)
   | 0.1783 (best: 0.2014) - {'k1': 2.9849239624654826, 'b': 0.6824238501196691}
 > iteration 7/40, took 0:00:07 (remains: 0:03:57)
   | 0.2017 (best: 0.2017) - {'k1': 1.0082228400685809, 'b': 0.5392847471129911}
 > iteration 8/40, took 0:00:07 (remains: 0:03:46)
   | 0.20

ConnectionTimeout: ConnectionTimeout caused by - ReadTimeoutError(HTTPConnectionPool(host='35.246.228.72', port=9200): Read timed out. (read timeout=1200))

In [7]:
for field, (_, _, _, metadata) in results.items():
    _ = plot_objective(metadata, sample_source='result')

NameError: name 'results' is not defined

In [ ]:
%%time

_ = for_all_fields(
    "Dev set evaluation",
    fn=lambda templates, similarity: evaluate_mrr100_dev_templated(es, max_concurrent_searches, index, templates, template_id, query_params),
    existing_results=results,
)

## Best parameters

In [ ]:
best_params = [(x, best) for x, (_, best, _, _) in results.items()]

In [ ]:
best_params

In [ ]:
print("Setting best params for each field")
set_bm25_params(best_params)

## Conclusion

If you'd like to reset all parameters to their defaults, you can do so with the following code. However if you want to stick with the optimal values as shown above, you should skip running this.

In [ ]:
reset_all()

In [ ]:
# best params from all previous runs with current fields and analyzers

[
    ('url', {'k1': 0.2835389588290694, 'b': 0.8307098387153782}), # 0.2187
    ('title', {'k1': 0.3477150744985997, 'b': 0.6174817900867441}), # 0.2349
    ('title.bigrams', {'k1': 1.2, 'b': 0.75}), # 0.1300
    ('body', {'k1': 3.0128735487205525, 'b': 0.8200709176657588}), # 0.2645
    ('body.bigrams', {'k1': 1.9241932055770454, 'b': 0.7257382745572979}), # 0.2041
    ('expansions', {'k1': 4.870954366799399, 'b': 0.9249613913608172}), # 0.3220
    ('expansions.bigrams', {'k1': 1.2, 'b': 0.75}) # 0.2837
]